# Calculate CESM Indeces

This file just calculates all the indeces for all months for all cesmFullForcingFiles.

Could be generalised if desired.

In [17]:
#import my functions
import helpers.fileHandler as fh
import utils._modelDefinitions as _model
import utils.sstIndex as sst
import utils.pslIndex as psl


In [2]:
import xarray

In [3]:
import warnings
warnings.filterwarnings('ignore')

Data for climatology

In [6]:
sstClimatDs=fh.loadModelData('CESM-LME', 'SST', '0850cntl.001')
pslClimatDs=fh.loadModelData('CESM-LME', 'PSL', '0850cntl.001')

sstClimat=sst.calculateClimatology(sstClimatDs, 850, 2005)
pslClimat=psl.calculateClimatology(pslClimatDs, 850, 2005)

#loading both those things
for keys in sstClimat:
    sstClimat[keys].load()
    
pslClimat.load()

CESM-LME/mon/SST/
b\.e11\.B.*?\.f19_g16\.0850cntl.001\..*?SST\..+\.nc
['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/SST/b.e11.B1850C5CN.f19_g16.0850cntl.001.pop.h.SST.085001-089912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/SST/b.e11.B1850C5CN.f19_g16.0850cntl.001.pop.h.SST.090001-099912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/SST/b.e11.B1850C5CN.f19_g16.0850cntl.001.pop.h.SST.100001-109912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/SST/b.e11.B1850C5CN.f19_g16.0850cntl.001.pop.h.SST.110001-119912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/SST/b.e11.B1850C5CN.f19_g16.0850cntl.001.pop.h.SST.120001-129912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/SST/b.e11.B1850C5CN.f19_g16.0850cntl.001.pop.h.SST.130001-139912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/SST/b.e11.B1850C5CN.f19_g16.0850cntl.001.pop.h.SST.140001-149912.nc', '/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/SST/b.e11.B1850C5CN.f19_g16.0850cntl.001.pop.h.SST.150001-159912.nc', '/Volumes/Untitle

<xarray.Dataset>
Dimensions:  (time: 13872)
Coordinates:
  * time     (time) object 0850-01-31 00:00:00 ... 2005-12-31 00:00:00
Data variables:
    lat1     (time) float32 1.015e+05 1.018e+05 ... 1.018e+05 1.016e+05
    lat2     (time) float32 9.869e+04 9.825e+04 ... 9.766e+04 9.842e+04

In [7]:
sstClimatDs.close()
pslClimatDs.close()

For each experiment in Full Forcings, calculate all the indeces

In [ ]:
for experiment in _model.cesmFullForcings:
    print(experiment)
    sstDs = fh.loadModelData('CESM-LME', 'SST', experiment)
    pslDs = fh.loadModelData('CESM-LME', 'PSL', experiment)
    pslIndex = psl.calculateSamIndex(pslDs, pslClimat)
    sstIndex = sst.calculateIndex(sstDs, sstClimat)
    indeces = xarray.merge([pslIndex, sstIndex])
    indeces.assign_attrs(climatology='calculated based on 850-2005 climatology from the control run')
    indeces.to_netcdf('results/cesmMonthlyIndeces/' + experiment + '.nc')


In [59]:
# RCP8.5 extensions are a special case


for experiment in ['002','003', '008', '009']:
    
    pre2005Ds = xarray.load_dataset('results/cesmMonthlyIndeces/' + experiment + '.nc')
                                    
    print('LME.'+experiment)
    
    sstDs = fh.loadModelData('CESM-LME', 'SST', 'LME\.'+experiment)
    
    extensionIndex=sst.calculateIndex(sstDs, sstClimat)
    
    sstIndex = xarray.concat(
        (pre2005Ds[[keys for keys in extensionIndex.data_vars]], extensionIndex),
        'time'
    )
        
    
    pslDs = fh.loadModelData('CESM-LME', 'PSL', 'LME.'+experiment)
        
    extensionIndex=psl.calculateSamIndex(pslDs, pslClimat)
    
    pslIndex=xarray.concat(
        (pre2005Ds[[keys for keys in extensionIndex.data_vars]], extensionIndex),
        'time'
    )
    
    indeces = xarray.merge([pslIndex, sstIndex])
    indeces.assign_attrs(climatology='calculated based on 850-1850 climatology')
    indeces.to_netcdf('results/cesmMonthlyIndeces/LME.' + experiment + '.nc')


LME.002
CESM-LME/mon/SST/
b\.e11\.B.*?\.f19_g16\.LME\.002\..*?SST\..+\.nc
['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/SST/b.e11.BRCP85C5CN.f19_g16.LME.002.pop.h.SST.200601-210012.nc']
1
False
['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/SST/b.e11.BRCP85C5CN.f19_g16.LME.002.pop.h.SST.200601-210012.nc']
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/SST/b.e11.BRCP85C5CN.f19_g16.LME.002.pop.h.SST.200601-210012.nc']
CESM-LME/mon/PSL/
b\.e11\.B.*?\.f19_g16\.LME.002\..*?PSL\..+\.nc
['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PSL/b.e11.BRCP85C5CN.f19_g16.LME.002.cam.h0.PSL.200601-210012.nc']
1
False
['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PSL/b.e11.BRCP85C5CN.f19_g16.LME.002.cam.h0.PSL.200601-210012.nc']
Files imported: 
 ['/Volumes/Untitled/CMIP5-PMIP3/CESM-LME/mon/PSL/b.e11.BRCP85C5CN.f19_g16.LME.002.cam.h0.PSL.200601-210012.nc']
